In [1]:
import os
# 원하는 디렉토리로 이동
os.chdir("/workspace")
# 현재 작업 디렉토리 확인
print("Current working directory:", os.getcwd())

Current working directory: /workspace


In [2]:
# Cell 1 : Import Libraries

import numpy as np
import os
import random
import math
import time
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import glob as gb
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from lightning.pytorch.utilities import CombinedLoader

# (src.model.models_cola) 안의 Cola, ColaMD 모델
from src.model.models_cola import Cola, ColaMD


from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import seed_everything
from sklearn.model_selection import train_test_split

from src.util import random_crop, random_mask, random_multiply, crop_first
from src.model.models_eval import AudioClassifier, AudioClassifierCLAP, AudioClassifierAudioMAE
from src.benchmark.model_util import get_encoder_path, initialize_pretrained_model
from src.util import train_test_split_from_list
import collections
from tqdm import tqdm
from pathlib import Path

torch.backends.cudnn.deterministic = True

In [3]:
labels_data = pd.read_csv('data/icbhi/ICBHI_Challenge_diagnosis.txt',
                          dtype=str, sep='\t', names=['userID', 'class'])
splits_data = pd.read_csv('data/icbhi/ICBHI_challenge_train_test.txt',
                          dtype=str, sep='\t', names=['fileID', 'group'])
demographics_data = pd.read_csv('data/icbhi/ICBHI_Challenge_demographic_information.txt',
                                dtype=str, sep='\t', names=['userId', 'Age', 'Sex', 'Adult_BMI', 'Child Weight', 'Child Height'])

BATCH_SIZE = 512
SR = 16000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

data_dir = "data/icbhi/"
feature_dir = "feature/icbhidisease_eval/"

audio_dir = data_dir + "ICBHI_final_database/"


def process_disease():
    sound_dir_loc = np.array(
        gb.glob("data/icbhi/ICBHI_final_database/*.wav"))
    labels = []
    filenames = []
    split = []

    for i in tqdm(range(sound_dir_loc.shape[0])):
        filename = sound_dir_loc[i].strip().split('.')[0]
        fileID = filename.split('/')[-1].split('.')[0]
        userID = filename.split('/')[-1].split('_')[0]
        disease_label = labels_data["class"][labels_data.userID ==
                                             userID].values[0]
        file_split = splits_data["group"][splits_data.fileID ==
                                          fileID].values[0]

        filenames.append(sound_dir_loc[i])
        labels.append(disease_label)
        split.append(file_split)

    np.save(feature_dir + "labels.npy", labels)
    np.save(feature_dir + "sound_dir_loc.npy", filenames)
    np.save(feature_dir + "split.npy", split)


def extract_and_save_embeddings(feature="operaCE", input_sec=8, dim=1280):
    sound_dir_loc = np.load(feature_dir + "sound_dir_loc.npy")

    from src.benchmark.model_util import extract_opera_feature
    opera_features = extract_opera_feature(
        sound_dir_loc, pretrain=feature, input_sec=input_sec, dim=dim)
    feature += str(dim)
    np.save(feature_dir + feature + "_feature.npy", np.array(opera_features))

In [4]:
pretrain = "operaCT"
dim = 768
min_len_htsat = 8

process_disease()

extract_and_save_embeddings(
    feature = "operaCT", input_sec=min_len_htsat, dim=dim)

100%|███████████████████████████████████████| 920/920 [00:00<00:00, 2523.08it/s]


extracting feature from operaCT model with input_sec 8


/opt/conda/envs/audio/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|███████████████████████████████████████| 920/920 [2:00:30<00:00,  7.86s/it]

(920, 768)


In [5]:



class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, data, max_len=256, augment=True, from_npy=False, crop_mode="first", from_audio=False):
        self.data = data[0]
        self.label = data[1]
        self.max_len = max_len
        self.augment = augment
        self.from_npy = from_npy
        self.crop_mode = crop_mode
        self.from_audio = from_audio

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.from_npy:
            npy_path = self.data[idx]
            x = np.load(npy_path + ".npy")
        else:
            x = self.data[idx]

        label = self.label[idx]

        if self.from_audio:
            return x, label

        if self.max_len:
            if self.crop_mode == "random":
                x = random_crop(x, crop_size=self.max_len)
            else:
                x = crop_first(x, crop_size=self.max_len)

        if self.augment:
            x = random_mask(x)
            x = random_multiply(x)

        x = torch.tensor(x, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)

        return x, label


class DecayLearningRate(pl.Callback):
    def __init__(self):
        self.old_lrs = []

    def on_train_start(self, trainer, pl_module):
        # track the initial learning rates
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            group = []
            for param_group in optimizer.param_groups:
                group.append(param_group["lr"])
            self.old_lrs.append(group)

    def on_train_epoch_end(self, trainer, pl_module):
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            old_lr_group = self.old_lrs[opt_idx]
            new_lr_group = []
            for p_idx, param_group in enumerate(optimizer.param_groups):
                old_lr = old_lr_group[p_idx]
                new_lr = old_lr * 0.99
                new_lr_group.append(new_lr)
                param_group["lr"] = new_lr
            self.old_lrs[opt_idx] = new_lr_group


In [6]:
def finetune_icbhidisease(n_cls=2, pretrain="operaCT", l2_strength=1e-4, epochs=64, batch_size=64, lr=1e-4, head="linear", feat_dim=1280):
    print("*" * 48)
    print("training dataset ICBHI disease from model pretrained on", pretrain, "with l2_strength", l2_strength, "lr", lr, "head", head)

    feature_dir = "feature/icbhidisease_eval/"

    from_audio = False
    if pretrain == 'operaCT':
        if not os.path.exists(feature_dir + "spectrogram_pad8.npy"):
            from src.util import get_split_signal_librosa
            sound_dir_loc = np.load(feature_dir + "sound_dir_loc.npy")
            x_data = []
            for audio_file in sound_dir_loc:
                #print(audio_file[:-4])
                #print(audio_file)
                data = get_split_signal_librosa("", audio_file[:-4], spectrogram=True, input_sec=8.18, trim_tail=False)[0]
                # print(data.shape)
                x_data.append(data)
            x_data = np.array(x_data)
            np.save(feature_dir + "spectrogram_pad8.npy", x_data)

        x_data = np.load(feature_dir + "spectrogram_pad8.npy")
        pretrained_model = initialize_pretrained_model(pretrain)
        if pretrain == "null":
            lr = 1e-4
            epochs = 64
            print("-" * 20 + "training from scratch")
        else:
            encoder_path = get_encoder_path(pretrain)
            print("loading weights from", encoder_path)
            ckpt = torch.load(encoder_path)
            pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)
        
        freeze_encoder = "early" if pretrain == "operaCE" else "none"
        net = pretrained_model.encoder
        #dim_out = pretrained_model.encoder.out_dim
        model = AudioClassifier(net=net, head=head, classes=n_cls, lr=lr, l2_strength=l2_strength, feat_dim=feat_dim, freeze_encoder=freeze_encoder)


    y_set = np.load(feature_dir + "split.npy")
    y_label = np.load(feature_dir + "labels.npy")
    print(collections.Counter(y_label))

    mask = (y_label == "Healthy") | (y_label == "COPD")
    y_label = y_label[mask]
    y_set = y_set[mask]
    x_data = x_data[mask]   

    label_dict = {"Healthy": 0, "COPD": 1}
    y_label = np.array([label_dict[y] for y in y_label])

    x_data_train, x_data_test, y_label_train, y_label_test = train_test_split_from_list(x_data, y_label, y_set)

    x_data_train, x_data_vad, y_label_train, y_label_vad = train_test_split(
        x_data_train, y_label_train, test_size=0.2, random_state=1337, stratify=y_label_train
    )

    print(collections.Counter(y_label_train))
    print(collections.Counter(y_label_vad))
    print(collections.Counter(y_label_test))

    train_data = AudioDataset((x_data_train, y_label_train), augment=False, max_len=False, from_audio=from_audio)
    test_data = AudioDataset((x_data_test, y_label_test), augment=False, max_len=False, from_audio=from_audio)
    val_data = AudioDataset((x_data_vad, y_label_vad), augment=False, max_len=False, from_audio=from_audio)

    train_loader = DataLoader(
        train_data, batch_size=batch_size, num_workers=2, shuffle=True
    )
    val_loader = DataLoader(
        val_data, batch_size=batch_size, num_workers=2, shuffle=True
    )
    test_loader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True, num_workers=2
    )

    logger = CSVLogger(
        save_dir="cks/logs/finetune", name="icbhi", 
        version="_".join([head, pretrain, str(batch_size), str(lr), str(epochs), str(l2_strength)])
    )

    checkpoint_callback = ModelCheckpoint(
        monitor="valid_auc", mode="max", dirpath="cks/finetune/icbhi/", 
        filename="_".join(["finetuning", head, pretrain, str(batch_size), str(lr), str(epochs), str(l2_strength)]) + "-{epoch:02d}-{valid_auc:.2f}",
        every_n_epochs=3,
    )

    trainer = pl.Trainer(
        max_epochs=epochs,
        accelerator="gpu",
        devices=1,
        # logger=logger,
        logger=False,
        callbacks=[DecayLearningRate(), checkpoint_callback],
        gradient_clip_val=1.0,
        log_every_n_steps=1,
        enable_progress_bar=False
    )
    trainer.fit(model, train_loader, val_loader)

    trainer.test(dataloaders=train_loader)
    trainer.test(dataloaders=val_loader)
    test_res = trainer.test(dataloaders=test_loader)
    auc = test_res[0]["test_auc"]
    print("finished training dataset icbhi disease from model pretrained on", pretrain,  "with l2_strength", l2_strength, "lr", lr, "head", head)
    return auc

### with Coswara

In [50]:


task = "icbhidisease"
pretrain = "operaCT"
gridsearch = False
lr = 1e-4
l2_strength = 1e-5
head = 'linear'
modality = 'cough'
mapgoogle = False
dim =1280
n_run = 5
label = 'smoker'
LOOCV = False
avgprob = False

if not LOOCV:
    auc_scores = []
    for seed in range(n_run):
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        seed_everything(seed, workers=True)
        if task == "icbhidisease":
            auc = finetune_icbhidisease(pretrain= pretrain, epochs=64, l2_strength=1e-4, feat_dim = 768)
        auc_scores.append(auc)
    print("=" * 48)
    print(auc_scores)
    print("Five times mean task {} finetuning from {} results: auc mean {:.3f} ± {:.3f}".format(task, pretrain, np.mean(auc_scores), np.std(auc_scores)) )
    print("=" * 48)

Seed set to 0


************************************************
training dataset ICBHI disease from model pretrained on operaCT with l2_strength 0.0001 lr 0.0001 head linear
loading weights from cks/model/combined/icbhi_icbhicycle_hf_lung_Coswara_kauh_PulmonarySound_SPRSound/encoder-test_experiment_transformer-epoch=49--valid_acc=0.83-valid_loss=0.7015.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


768 2
Counter({'COPD': 793, 'Pneumonia': 37, 'Healthy': 35, 'URTI': 23, 'Bronchiectasis': 16, 'Bronchiolitis': 13, 'LRTI': 2, 'Asthma': 1})
(828, 256, 64) 828 828
Counter({1: 355, 0: 14})
Counter({1: 89, 0: 4})
Counter({1: 349, 0: 17})



  | Name | Type             | Params
------------------------------------------
0 | net  | EncoderHTSAT     | 31.3 M
1 | head | Sequential       | 1.5 K 
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
30.2 M    Trainable params
1.1 M     Non-trainable params
31.3 M    Total params
125.307   Total estimated model params size (MB)
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
`Trainer.fit` stopped: `max_epochs=64` reached.
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If

...

test_auc tensor(1.)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_auc                    1.0
        test_loss          0.0002993276866618544
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Restoring states from the checkpoint path at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=11-valid_auc=1.00-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=11-valid_auc=1.00-v1.ckpt


test_auc tensor(1.)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9892473118279571
        test_auc                    1.0
        test_loss          0.061083901673555374
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Restoring states from the checkpoint path at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=11-valid_auc=1.00-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=11-valid_auc=1.00-v1.ckpt


test_auc tensor(0.9553)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9617486338797814
        test_auc            0.9553345441818237
        test_loss           0.17229753732681274
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
finished training dataset icbhi disease from model pretrained on operaCT with l2_strength 0.0001 lr 0.0001 head linear
[0.949266791343689, 0.9575256705284119, 0.9448845386505127, 0.9625822305679321, 0.9553345441818237]
Five times mean task icbhidisease finetuning from operaCT results: auc mean 0.954 ± 0.006


In [ ]:
encoder-experiment_wo Coswara_kauh_Pul_20_SPR_20-epoch=129--valid_acc=0.77-valid_loss=0.8953

In [42]:
ckpt = torch.load("cks/model/combined/icbhi_icbhicycle_hf_lung_Coswara_kauh_PulmonarySound_SPRSound/encoder-test_experiment_transformer-epoch=49--valid_acc=0.83-valid_loss=0.7015.ckpt")
pretrained_model = initialize_pretrained_model(pretrain)
pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)

<All keys matched successfully>

In [44]:
print(pretrained_model)

Cola(
  (do): Dropout(p=0.1, inplace=False)
  (encoder): EncoderHTSAT(
    (encoder): HTSATWrapper(
      (htsat): HTSAT_Swin_Transformer(
        (spectrogram_extractor): Spectrogram(
          (stft): STFT(
            (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
            (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
          )
        )
        (logmel_extractor): LogmelFilterBank()
        (spec_augmenter): SpecAugmentation(
          (time_dropper): DropStripes()
          (freq_dropper): DropStripes()
        )
        (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (patch_embed): PatchEmbed(
          (proj): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        )
        (pos_drop): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0): BasicLayer(
            dim=96, input_r

### without Coswara

In [8]:
import argparse
from pathlib import Path

task = "icbhidisease"
pretrain = "operaCT"
gridsearch = False
lr = 1e-4
l2_strength = 1e-5
head = 'linear'
modality = 'cough'
mapgoogle = False
dim =1280
n_run = 5
label = 'smoker'
LOOCV = False
avgprob = False

if not LOOCV:
    auc_scores = []
    for seed in range(n_run):
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        seed_everything(seed, workers=True)
        if task == "icbhidisease":
            auc = finetune_icbhidisease(pretrain= pretrain, epochs=64, l2_strength=1e-4, feat_dim = 768)
        auc_scores.append(auc)
    print("=" * 48)
    print(auc_scores)
    print("Five times mean task {} finetuning from {} results: auc mean {:.3f} ± {:.3f}".format(task, pretrain, np.mean(auc_scores), np.std(auc_scores)) )
    print("=" * 48)

Seed set to 0


************************************************
training dataset ICBHI disease from model pretrained on operaCT with l2_strength 0.0001 lr 0.0001 head linear
loading weights from cks/model/combined/icbhi_icbhicycle_hf_lung_kauh_PulmonarySound_SPRSound/encoder-experiment_wo Coswara_kauh_Pul_20_SPR_20-epoch=129--valid_acc=0.77-valid_loss=0.8953.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /workspace/cks/finetune/icbhi exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

  | Name | Type             | Params
------------------------------------------
0 | net  | EncoderHTSAT     | 31.3 M
1 | head | Sequential       | 1.5 K 
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
30.2 M    Trainable params
1.1 M     Non-trainable params
31.3 M    Total params
125.307   Total estimated model params size (MB)


768 2
Counter({'COPD': 793, 'Pneumonia': 37, 'Healthy': 35, 'URTI': 23, 'Bronchiectasis': 16, 'Bronchiolitis': 13, 'LRTI': 2, 'Asthma': 1})
(828, 256, 64) 828 828
Counter({1: 355, 0: 14})
Counter({1: 89, 0: 4})
Counter({1: 349, 0: 17})


/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
`Trainer.fit` stopped: `max_epochs=64` reached.
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=05-valid_auc=1.00-v6.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.000

...

test_auc tensor(0.9960)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.986449864498645
        test_auc            0.9959758520126343
        test_loss           0.03364144638180733
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Restoring states from the checkpoint path at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=02-valid_auc=1.00-v10.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=02-valid_auc=1.00-v10.ckpt


test_auc tensor(1.)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9784946236559141
        test_auc                    1.0
        test_loss           0.04362513870000839
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Restoring states from the checkpoint path at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=02-valid_auc=1.00-v10.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/icbhi/finetuning_linear_operaCT_64_0.0001_64_0.0001-epoch=02-valid_auc=1.00-v10.ckpt


test_auc tensor(0.9675)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9590163934426229
        test_auc            0.9674700498580933
        test_loss           0.12098512053489685
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
finished training dataset icbhi disease from model pretrained on operaCT with l2_strength 0.0001 lr 0.0001 head linear
[0.9831451177597046, 0.957188606262207, 0.982639491558075, 0.9565144777297974, 0.9674700498580933]
Five times mean task icbhidisease finetuning from operaCT results: auc mean 0.969 ± 0.012
